In [1]:
import pickle
import pandas as pd
import numpy as np
import re
import string

In [2]:
from nltk.stem.snowball import SnowballStemmer
import pymorphy2

In [3]:
stemmer = SnowballStemmer("russian") 
morph = pymorphy2.MorphAnalyzer()

In [4]:
# вытаскивает топ50 сырых фитчей
def get_features(vector, model, vec_model, k):
    
    vector = vector.toarray()[0]
    names = vec_model.get_feature_names_out()
    coefs = model.coef_
    
    not_null_values = [(i, v, coefs[i]) for i, v in enumerate(vector) if v != 0]
    top_100 = sorted(not_null_values, key=lambda x: abs(x[2]), reverse=True)[:100]
    words = [names[ind] for ind, tf_idf, c in top_100]
    adj = [w for w in words if re.search(r'\w+[ыио]й(?!\w)', w)]
    
    if len(adj) >= k*(2/3):
        res = adj[:round(k*(2/3))] + [w for w in words if w not in adj][:round(k/3)]
    else:
        res = adj + [w for w in words if w not in adj][:k-len(adj)]
        
    return res

In [5]:
# нормализация одного слова
def normalized(word):
    word_no_punct = word.translate(str.maketrans('', '', string.punctuation + "«»—№–"))
    word_lem = morph.parse(word.lower())[0].normal_form
    word_stem = stemmer.stem(word_lem)
    return stemmer.stem(word_no_punct.lower())

In [6]:
# возвращение сырых слов из основго предложения
def return_important_substrings(sentence, features):
    
    result = []
    for feature in features:
        flag = False
        f = [stemmer.stem(_) for _ in feature.split()]
        sequence = []

        for token in sentence.split():
            t = normalized(token)

            # True - начать собирать последовательность
            if t.startswith(f[0]) or f[0] in t:

                if flag:
                    flag = False
                    sequence.clear()
                else:
                    flag = True

            if flag:
                sequence.append(token)

                if f[-1] in t or len(sequence) > 6:
                    flag = False
                    break

        seq = ' '.join(sequence)
        if seq:
            result.append(seq)
            
    return result

In [ ]:
features = get_features(VECTOR, model, vec_model, 40)
result = return_important_substrings(YOURSTRING, features)

#### Example

In [7]:
df = pd.read_csv('datasets/dataset.csv').drop(columns='Unnamed: 0')
df.tail()

,pr_txt,Категория,Уровень рейтинга,preprocessed_text,target,scaled_target
3595,«Эксперт РА» подтвердило рейтинг «Татнефти» на...,AAA,AAA,подтвердить уровень москва апрель рейтинговый ...,9,0.95
3596,«Эксперт РА» понизило рейтинг компании «Инвест...,BB,BB,понизить уровень изменить прогноз установить с...,-3,0.35
3597,АКРА подтвердило кредитный рейтинг ПАО «КАМАЗ»...,A,A+,подтвердить кредитный прогноз также подтвердит...,4,0.70
3598,АКРА подтвердило кредитный рейтинг ФГУП «Почта...,AAA,AAA,подтвердить кредитный фгуп уровень прогноз кре...,9,0.95
3599,«Эксперт РА» подтвердил рейтинг компании СИБИН...,BB,BB+,подтвердить сибинтек уровень москва апрель рей...,-2,0.40


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
vec_model = TfidfVectorizer(ngram_range=(3, 3))

X_train = vec_model.fit_transform(df['preprocessed_text'][:round(df.shape[0]*0.7)])
X_test = vec_model.transform([df['preprocessed_text'][df.shape[0]-1]])

In [10]:
sentence = df['pr_txt'][df.shape[0]-1]
sentence[:500]

'«Эксперт РА» подтвердил рейтинг компании СИБИНТЕК на уровне <Рейтинг>. Москва, 18 апреля 2019 г. Рейтинговое агентство «Эксперт РА» подтвердило рейтинг нефинансовой компании «СИБИНТЕК» на уровне «Рейтинг». Прогноз по рейтингу – стабильный. ООО ИК «СИБИНТЕК» предоставляет ИТ-услуги компаниям Группы «Роснефть» и ряду сторонних заказчиков. В текущем портфеле заказов, проектов и услуг для компаний Группы «Роснефть» в сфере ИТ автоматизация АСУ ТП занимает значительную часть – по оценкам агентства, п'

In [11]:
vector = X_test
vector

<1x529090 sparse matrix of type '<class 'numpy.float64'>'
	with 758 stored elements in Compressed Sparse Row format>

In [12]:
with open(r'models/LinReg.pkl', 'rb') as model:
    model = pickle.load(model)

C:\Users\justa\anaconda3\lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.0.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [13]:
features = get_features(vector, model, vec_model, 40)
result = return_important_substrings(sentence, features)

In [14]:
result

['привлечь дополнительное внешнее',
 'кредитных линий, имеющихся в долгосрочной перспективе, умеренно',
 'социального страхования РФ,',
 'объединения информационных ресурсов,',
 'связано, в том числе, с наличием в',
 'программного обеспечения в иностранной',
 'отчетная дата) задолженность компании составила около 5',
 'года около 86% выручки происходит от этих',
 'отдельного подразделения по управлению',
 'представил членам рейтингового',
 'июня 2018 года. Предыдущий',
 'объектах',
 'часть – по оценкам агентства, по итогам',
 'мониторинга промышленных объектов',
 'предназначенный для перехода к облачной',
 'Рейтинговое агентство «Эксперт РА» подтвердило рейтинг нефинансовой',
 'среди компаний, специализирующихся на информационных',
 'пересмотрена (в рамках изменений в учетной политике',
 'кредиторской задолженности. По состоянию',
 'агентство «Эксперт РА» подтвердило рейтинг нефинансовой',
 'предварительного уведомления, если иное',
 'реализации данного проекта,',
 'управленческая отче